<a href="https://colab.research.google.com/github/Arijit-datascience/TinyImagenet200/blob/main/Tinyimagenet200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

from tqdm import notebook
from PIL import Image
import requests
from io import BytesIO
import glob
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%%capture
!git clone https://github.com/Arijit-datascience/pytorch_cifar10.git
!pip install albumentations==1.0.0

In [ ]:
from pytorch_cifar10 import main as main
from pytorch_cifar10.model import resnet as models
from pytorch_cifar10.utils import utils as utils
from pytorch_cifar10.utils.tiny_dataloader import get_id_dictionary, get_data, ImagenetDataset, get_dataloaders, calculate_mean_std, get_transforms

## Check if Tiny Imagenet data is already downloaded

In [ ]:
#Set the dataset folder path
dataset_path = "/gdrive/MyDrive/data/tiny-imagenet-200/"

if not os.path.isdir(dataset_path):
  r = requests.get('http://cs231n.stanford.edu/tiny-imagenet-200.zip', stream=True)
  print('Downloading TinyImageNet Data...')
  zip_ref = zipfile.ZipFile(BytesIO(r.content))
  for file in notebook.tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
      zip_ref.extract(member=file, path='/gdrive/MyDrive/data/')
  zip_ref.close()
else:
  print("TinyImageNet Data Already downloaded")

TinyImageNet Data Already downloaded


In [ ]:
X_train, X_test, y_train, y_test = get_data(dataset_path)

starting loading data
Training samples: 70000
Testing samples: 30000
Data loading completed, in 0.13709282875061035 seconds


In [ ]:
mean,std = calculate_mean_std()
train_transform, test_transform = get_transforms(mean,std)

# Extract and transform the data
train_loader,test_loader = get_dataloaders(X_train, X_test, y_train, y_test, train_transform, test_transform)

In [ ]:
# Model summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = models.ResNet18("BN").to(device)
summary(model, input_size=(3, 64, 64))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 64, 64]          73,728
      BatchNorm2d-14          [-1,

In [ ]:
%%capture
!pip install torch-lr-finder
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.002)
criterion = torch.nn.CrossEntropyLoss()
from pytorch_cifar10.utils import lr_finder

In [ ]:
from torch_lr_finder import LRFinder


def find_lr(net, optimizer, criterion, train_loader):
    """Find learning rate for using One Cyclic LRFinder
    Args:
        net (instace): torch instace of defined model
        optimizer (instance): optimizer to be used
        criterion (instance): criterion to be used for calculating loss
        train_loader (instance): torch dataloader instace for trainig set
    """
    lr_finder = LRFinder(net, optimizer, criterion, device="cuda")
    lr_finder.range_test(train_loader, end_lr=10, num_iter=100, step_mode="exp")
    lr_finder.plot()
    lr_finder.reset()

In [ ]:
import torch.nn as nn

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_loader, end_lr=10, num_iter=100, step_mode="exp")
lr_finder.plot()
lr_finder.reset()

error: ignored

In [ ]:
EPOCH = 24
MAX_EPOCH=5
batch_size = 512
l1_factor = 0
l2_factor = 0.0001
lr_value = 0.001
mt_val = 0.9

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

utils.seed_everything(1)

model = models.ResNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.002)
scheduler = OneCycleLR(optimizer, max_lr=max_lr, steps_per_epoch=len(train_loader), epochs=EPOCH, pct_start=(MAX_EPOCH/EPOCH), div_factor=8)

print("------------------------------------------")
train_loss, test_loss, train_acc, test_acc = main.main(EPOCH, model, device, train_loader, test_loader, optimizer, scheduler, l1_factor)